In [2]:
import pandas as pd
import numpy as np
import json
from datetime import date, timedelta
#from sodapy import Socrata
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import urllib.request
from selenium.webdriver.common.keys import Keys

In [2]:
date_2year_ago = (date.today() - timedelta(days=2*365))
date_today = date.today()
print('Las fechas a utilizar son: fecha dos anios atras: ' + str(date_2year_ago) + ' y fecha actual: ' + str(date_today))
# Cambiar esta variable si se necesita una fecha especifica
date_2year_ago = 20190000#int(date_2year_ago.strftime('%Y%m%d'))
date_today = int(date_today.strftime('%Y%m%d'))

Las fechas a utilizar son: fecha dos anios atras: 2019-04-23 y fecha actual: 2021-04-22


In [3]:
def first_df():
    # leemos desde la URL el json que contiene la tabla de informacion acerca de las tasas
    df = pd.read_json('https://www.datos.gov.co/resource/mcec-87by.json')
    # hacemos transformaciones a las fechas para tenerlas YYYYMMDD numericas
    df['vigenciadesde'] = pd.to_datetime(df['vigenciadesde']).dt.strftime('%Y%m%d').astype('int32')
    df['vigenciahasta'] = pd.to_datetime(df['vigenciahasta']).dt.strftime('%Y%m%d').astype('int32')
    # retornamos el data frame desde la fecha 2019
    return df[df['vigenciadesde'] >= date_2year_ago]

In [4]:
df  = first_df()
df

,valor,unidad,vigenciadesde,vigenciahasta
0,3630.81,COP,20210423,20210423
1,3639.12,COP,20210422,20210422
2,3636.26,COP,20210421,20210421
3,3606.42,COP,20210420,20210420
4,3595.57,COP,20210417,20210419
...,...,...,...,...
545,3128.07,COP,20190110,20190110
546,3164.75,COP,20190109,20190109
547,3208.56,COP,20190105,20190108
548,3241.20,COP,20190104,20190104


In [5]:
# Se agrupa por anio y mes. Se cuenta para saber cuantos registros hay por cada mes lo que equivale
# a la cantidad de dias por mes
tasa_spot = first_df()
periodo = (tasa_spot['vigenciadesde']/100).astype('int32')
data = periodo.to_list()
names = ['periodo']
new_df = pd.DataFrame(data, columns=names)
new_df.value_counts().sort_index()

periodo
201901     19
201902     19
201903     21
201904     19
201905     21
201906     19
201907     20
201908     21
201909     19
201910     22
201911     19
201912     20
202001     19
202002     20
202003     20
202004     20
202005     20
202006     18
202007     22
202008     19
202009     21
202010     22
202011     16
202012     21
202101     18
202102     19
202103     21
202104     15
dtype: int64

In [6]:
# Enter the information from those sections here
#socrata_domain = 'datos.gov.co'
#socrata_dataset_identifier = 'mcec-87by'

# App Tokens can be generated by creating an account at https://opendata.socrata.com/signup
# Tokens are optional (`None` can be used instead), though requests will be rate limited.
#
# If you choose to use a token, run the following command on the terminal (or add it to your .bashrc)
# $ export SODAPY_APPTOKEN=<token>
#credentials = json.load(open('credentials.json'))
#socrata_token = credentials['TOKEN']


In [7]:
#client = Socrata(socrata_domain, socrata_token)
#print("Domain: {domain:}\nSession: {session:}\nURI Prefix: {uri_prefix:}".format(**client.__dict__))

In [8]:
#results = client.get(socrata_dataset_identifier)
#df = pd.DataFrame.from_dict(results)
#df_splt = df['vigenciadesde'].str.split('-')
#names = ['anio', 'mes', 'dia']
#data2 = df_splt.to_list()
#new_df2 = pd.DataFrame(data2, columns=names)
#new_df2.groupby(['anio', 'mes']).count()

In [9]:
# Segundo dataframe
# Se configuran las opciones del navegador para que empiece maximixado y sin extensiones habilitadas
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

In [10]:
# Funcion para eliminar duplicados de una lista
def remove_duplicate_list(lst):
    ''' Funcion para eliminar duplicados de una lista'''
    return list(dict.fromkeys(lst))

# Se crea instancia del driver que abrira el navegador web con las opciones definidas anteriormente
# Cambiar direccion a donde se encuentra el driver
driver_path = r'C:\Users\Efra\Downloads\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(driver_path, options = options)
# Se le pasa el link a abrir en el navegador
driver.get('https://totoro.banrep.gov.co/analytics/saw.dll?Portal&PortalPath=%2Fshared%2FDashboards_T%2FD_Estad%C3%ADsticas%2FEstad%C3%ADsticas&NQUser=publico&NQPassword=publico123&lang=es&page=Tasas%20de%20inter%C3%A9s%20y%20sector%20financiero&pagina=Otras%20tasas%20de%20inter%C3%A9s')

WebDriverWait(driver, 15)
driver.switch_to.frame(driver.find_element_by_id('frame_dashboard'))

# Se da click en el boton otras tasas de interes
WebDriverWait(driver, 15)\
.until(EC.element_to_be_clickable((By.XPATH,
                                      '/html/body/div[7]/div/div/table/tbody/tr/td[1]/div/div/table/tbody/tr/td[2]/div/table/tbody/tr/td[2]/table/tbody/tr/td/div')))\
    .click()

# Se da click en el menu desplegable Indicador bancario de referencia IBR
WebDriverWait(driver, 15)\
.until(EC.element_to_be_clickable((By.CLASS_NAME,
                                      'sui-treeview-item-toggle-collapsed')))
driver.find_elements_by_class_name('sui-treeview-item-toggle-collapsed')[1].click()


#print(len(driver.find_elements_by_xpath("//a[@href]")))
# Se guardan todos los links asocaidos al atributo o etiqueta href
href = driver.find_elements_by_xpath("//a[@href]")
href_list = []
# Se recorre la lista href en busca de los links que pertenezcan a los insumos de IBR
for i in href:
    try:
        contains = str(i.get_attribute('href')).replace('%20', '').upper()
        #print(contains)
        # Si el href contiene alguno de los strings, es guardado en una lista que contendra los enlaces a descargar
        if('IBR_PLAZOOVERNIGHTNOMINALPARAUNRANGODEFECHASDADOIQY' in contains 
           or 'IBR_PLAZOUNMESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOTRESMESESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOMESVENCIDONOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOSEISMESESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           and 'DOWNLOAD' in contains):
            src = i.get_attribute('href')
            href_list.append(src)
            #print('saca algo ' + src)
    except:
        #print(error)
        print('error href')

# Como la lista posiblemente guarde enlaces duplicados, se aplica la funcion para remover duplicados y asi descargar una sola vez
# cada archivo
href_list = remove_duplicate_list(href_list)
print('Cantidad de archivos a descargar: ' + str(len(href_list)))
for j in href_list:
    try:
        driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 't') 
        driver.get(j)
        print('Archivo descargado asociado a: ' + j)
    except AssertionError as error:
        print(error)
        print('error link')
        
# Cambiar a 60 segundos para que descargue antes de cerrar
time.sleep(30)
driver.close()

error href
error href
error href
error href
Cantidad de archivos a descargar: 5
Archivo descargado asociado a: https://totoro.banrep.gov.co/analytics/saw.dll?Download&Format=excel2007&Extension=.xls&BypassCache=true&lang=es&path=%2Fshared%2FSeries%20Estad%C3%ADsticas_T%2F1.%20IBR%2F%201.1.IBR_Plazo%20overnight%20nominal%20para%20un%20rango%20de%20fechas%20dado%20IQY
Archivo descargado asociado a: https://totoro.banrep.gov.co/analytics/saw.dll?Download&Format=excel2007&Extension=.xls&BypassCache=true&lang=es&path=%2Fshared%2FSeries%20Estad%C3%ADsticas_T%2F1.%20IBR%2F%201.2.IBR_Plazo%20un%20mes%20nominal%20para%20un%20rango%20de%20fechas%20dado%20IQY
Archivo descargado asociado a: https://totoro.banrep.gov.co/analytics/saw.dll?Download&Format=excel2007&Extension=.xls&BypassCache=true&lang=es&path=%2Fshared%2FSeries%20Estad%C3%ADsticas_T%2F1.%20IBR%2F%201.3.IBR_Plazo%20tres%20meses%20nominal%20para%20un%20rango%20de%20fechas%20dado%20IQY
Archivo descargado asociado a: https://totoro.banre

In [6]:
# cambiar ruta de descarga a la que se usara ex: C:\Users\Efra\Downloads
def clean_ibr_df():
    # Limpieza y adecuacion primer data frame IBR plazo overnight
    df_overnight = pd.read_excel(r'C:\Users\Efra\Downloads\1.1.IBR_Plazo overnight nominal para un rango de fechas dado IQY.xlsx')
    df_overnight.columns = df_overnight.iloc[7]
    df_overnight.drop(df_overnight.index[0:8], inplace = True)
    df_overnight.reset_index(drop = True, inplace = True)
    df_overnight.rename(columns = {df_overnight.columns[0] : 'fecha', df_overnight.columns[3] : 'ibr_efectivo'}, inplace = True)
    #df_overnight.drop(df_overnight.columns.difference(['fecha','ibr_efectivo']), 1, inplace=True)
    df_overnight.dropna()
    return df_overnight.head(10)
clean_ibr_df()

7,fecha,Excepción,Plazo real (días calendario),ibr_efectivo,ibr_efectivo,BANCO DAVIVIENDA,Colpatria Red Multibanca,BANAGRARIO,BANCO OCCIDENTE,Helm Bank S.A.,...,Banco Gnb Sudameris S.A.,CITIBANK,The Royal Bank of Scotland S.A.,Scotiabank Colombia S.A,BANCOLOMBIA,ITAU CORPBANCA,Banco Santander Colombia,BANCO CORPBANCA,Banco Popular S.A.,BANCO DE BOGOTÁ
0,2021-04-22 00:00:00,N,1,"1,747","1,708","1,711",NaN,"1,686","1,710",NaN,...,"1,689",NaN,NaN,NaN,"1,703","1,706",NaN,NaN,NaN,"1,711"
1,2021-04-21 00:00:00,N,1,"1,742","1,703","1,706",NaN,"1,685","1,697",NaN,...,"1,686",NaN,NaN,NaN,"1,700","1,706",NaN,NaN,NaN,"1,711"
2,2021-04-20 00:00:00,N,1,"1,739","1,700","1,708",NaN,"1,687","1,699",NaN,...,"1,684",NaN,NaN,NaN,"1,697","1,701",NaN,NaN,NaN,"1,711"
3,2021-04-19 00:00:00,N,1,"1,739","1,700","1,702",NaN,"1,688","1,711",NaN,...,"1,683",NaN,NaN,NaN,"1,695","1,698",NaN,NaN,NaN,"1,711"
4,2021-04-16 00:00:00,N,3,"1,731","1,693","1,702",NaN,"1,686","1,687",NaN,...,"1,681",NaN,NaN,NaN,"1,685","1,699",NaN,NaN,NaN,"1,711"
5,2021-04-15 00:00:00,N,1,"1,731","1,693","1,702",NaN,"1,685","1,687",NaN,...,"1,671",NaN,NaN,NaN,"1,680","1,699",NaN,NaN,NaN,"1,711"
6,2021-04-14 00:00:00,N,1,"1,734","1,696","1,695",NaN,"1,682","1,705",NaN,...,"1,679",NaN,NaN,NaN,"1,670","1,697",NaN,NaN,NaN,"1,711"
7,2021-04-13 00:00:00,N,1,"1,730","1,692","1,693",NaN,"1,680","1,600",NaN,...,"1,676",NaN,NaN,NaN,"1,690","1,695",NaN,NaN,NaN,"1,711"
8,2021-04-12 00:00:00,N,1,"1,726","1,688","1,696",NaN,"1,685","1,660",NaN,...,"1,671",NaN,NaN,NaN,"1,680","1,698",NaN,NaN,NaN,"1,711"
9,2021-04-09 00:00:00,N,3,"1,734","1,696","1,697",NaN,"1,680","1,711",NaN,...,"1,669",NaN,NaN,NaN,"1,669","1,696",NaN,NaN,NaN,"1,705"


In [6]:
pd.set_option('display.max_rows', None)
# cambiar ruta de descarga a la que se usara ex: C:\Users\Efra\Downloads
def clean_ibr_df():
    # Limpieza y adecuacion primer data frame IBR plazo overnight
    #df_overnight = pd.read_excel(r'C:\Users\Efra\Downloads\1.1.IBR_Plazo overnight nominal para un rango de fechas dado IQY.xlsx', header = 8, skipfooter = 7)
    df_overnight = pd.read_excel(r'C:\Users\Efra\Downloads\1.4.IBR_Plazo mes vencido nominal para un rango de fechas dado IQY.xlsx', header = 8, skipfooter = 7)
    df_overnight.rename(columns = {'Fecha (dd/mm/aaaa)' : 'Fecha', 'IBR' : 'IBR_efectivo', 'IBR.1' : 'IBR_nominal'}, inplace = True)
    df_overnight = df_overnight[['Fecha', 'IBR_efectivo', 'IBR_nominal']]
    df_overnight['Fecha'] = pd.to_datetime(df_overnight['Fecha']).dt.strftime('%Y%m%d').astype('int32')
    #df_overnight.dropna(subset = ['IBR_efectivo', 'IBR_nominal'], thresh=2, inplace = True)
    
    df_overnight.IBR_efectivo.replace({',':'.'}, regex = True, inplace = True)
    df_overnight['IBR_efectivo'] = df_overnight['IBR_efectivo'].astype(float)
    df_overnight.IBR_nominal.replace({',':'.'}, regex = True, inplace = True)
    df_overnight['IBR_nominal'] = df_overnight['IBR_nominal'].astype(float)
    
    df_overnight['IBR_efectivo'].fillna(0, inplace = True)
    df_overnight['IBR_nominal'].fillna(0, inplace = True)

    return df_overnight
clean_ibr_df()
# cuarto DF tomar IBR.1 si IBR esta vacio y aplicar formula para cambiar a tasa efectiva

,Fecha,IBR_efectivo,IBR_nominal
0,20120731,0.0,5.015
1,20120724,0.0,5.161
2,20120717,0.0,5.160
3,20120710,0.0,5.145
4,20120703,0.0,5.098
5,20120626,0.0,5.101
6,20120619,0.0,5.088
7,20120612,0.0,5.088
8,20120605,0.0,5.080
9,20120529,0.0,5.076


In [71]:
r = clean_ibr_df()
r.dtypes

Fecha             int32
IBR_efectivo    float64
IBR_nominal     float64
dtype: object

In [72]:
r.describe()

,Fecha,IBR_efectivo,IBR_nominal
count,2.390000e+02,0.0,239.000000
mean,2.009888e+07,NaN,5.383561
std,1.336801e+04,NaN,2.419685
min,2.008011e+07,NaN,2.950000
25%,2.009026e+07,NaN,3.295000
50%,2.010042e+07,NaN,4.450000
75%,2.011061e+07,NaN,8.087500
max,2.012073e+07,NaN,9.512000
